In [1]:
import cv2
import numpy as np
import os
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import cm
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd

from skimage import io, color
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches



In [2]:
!pip install opencv-python
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn = nn.Sequential(nn.Conv2d(1,3,kernel_size=2, padding=0,stride=2),
                                 nn.ReLU(),
                                nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                nn.ReLU(),
                                nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                nn.ReLU(),
                                nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                nn.ReLU(),
                                nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                nn.ReLU(),
                                nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                nn.ReLU(),
                                nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(128),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(64),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(32),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(16),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(16, 9, kernel_size=3, stride=1, padding=1),
                                nn.BatchNorm2d(9),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(9, 2, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2))
    
                                
                                 
    def forward(self, input):
        output = self.cnn(input)
        #         mean_chrominance = self.regressor(output)

        return  output

In [4]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev) 

In [5]:
# Model
model = CNN()
model.to(device = device)

CNN(
  (cnn): Sequential(
    (0): Conv2d(1, 3, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (7): ReLU()
    (8): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (9): ReLU()
    (10): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (11): ReLU()
    (12): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Upsample(scale_factor=2.0, mode=nearest)
    (16): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (18): ReLU()
    (19): Upsample(scale_factor=2.0, mode=nearest)
    (20): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padd

In [6]:

model = torch.load("./colorizer_250.pt")

In [7]:
# Hyperparameters
learning_rate = 1e-7
batch_size = 10

In [309]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

# NCD

In [120]:
df_ssims = pd.DataFrame()

In [263]:
classes = os.listdir("./Dataset/ColorfulOriginal")
classes.sort()
classes[7]

['.DS_Store',
 'Apple',
 'Banana',
 'Brinjal',
 'Broccoli',
 'CapsicumGreen',
 'Carrot',
 'Cherry',
 'ChilliGreen',
 'Corn',
 'Cucumber',
 'LadyFinger',
 'Lemon',
 'Orange',
 'Peach',
 'Pear',
 'Plum',
 'Pomegranate',
 'Potato',
 'Strawberry',
 'Tomato']

In [122]:
## (Apple)Average SSIM = <br>

In [520]:
ncd_test = glob.glob(os.path.join("./Dataset/Gray/Banana/","*.jpg"))
ncd_test.sort()
ncd_original = glob.glob(os.path.join("./Dataset/ColorfulOriginal/Banana/","*.jpg"))
ncd_original.sort()

In [521]:
class NCDTestDataset(Dataset):
    def __init__(self):
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.annotations = ncd_test
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = ncd_test[index]
        img_path_o = ncd_original[index]
        T = transforms.Resize((128, 128))
        image = Image.open(img_path)
        image_o = Image.open(img_path_o)
        image = T(image)
        image_o = T(image_o)
        image = self.transform(image)
        image_o = self.transform(image_o)
        return (image, image_o)

In [522]:
ncd_test_loader = DataLoader(NCDTestDataset(), batch_size=batch_size, shuffle=True)

In [523]:
for b, (image, image_o) in enumerate(ncd_test_loader):
    image = image.to(device = device)
    pred = model(image)
    
    save_predict_image_ncd(pred.cpu(), image.cpu(), b, image_o.cpu())

<Figure size 432x288 with 0 Axes>

In [514]:
from skimage.metrics import structural_similarity as ssim

In [515]:
def to_rgb_ncd(grayscale_input, ab_input, original, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
    Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).detach().numpy() # combine channels
    color_image = color_image + 1
    color_image = color_image - color_image.min()
    color_image = color_image / (color_image.max() - color_image.min())
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
    color_image = color.lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    original = original.numpy()
    original = original.transpose()

    #plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path + "grayscale", save_name), cmap='gray')
    plt.imsave(arr=original, fname='{}{}'.format(save_path, save_name))
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path, save_name))

In [519]:
def save_predict_image_ncd(pred, data, batch_idx, image_o):
    for i in range(len(data)):
        name = str(batch_idx) + " " + str(i)
        to_rgb_ncd(data[i], pred[i], image_o[i], save_path = "./ncd_predicted_images/Banana/", save_name = f"{name}.jpg")

In [517]:
lists = os.listdir("./Dataset/ColorfulOriginal/Banana")
lists.sort()
pred_list = os.listdir("./ncd_predicted_images/Banana")
pred_list.sort()


In [518]:
import torchvision.transforms as T
transform = T.Resize((128,128))
ssim_vals = []
for i in range(len(pred_list)):
    a = Image.open(os.path.join("./Dataset/ColorfulOriginal/Banana",lists[i]))
    a = transform(a)
    a = np.array(a)
    b = Image.open(os.path.join("./ncd_predicted_images/Banana",pred_list[i]))
    b = transform(b)
    b = np.array(b) 
    ssim_vals.append(ssim(a, b, channel_axis = -1))

IsADirectoryError: [Errno 21] Is a directory: './ncd_predicted_images/Banana/.ipynb_checkpoints'

In [504]:
len(ssim_vals)

48

In [505]:
df = pd.DataFrame()
df["Carrot"] = ssim_vals
df_final = pd.concat([df,df_final], ignore_index=True, axis=1)

In [506]:
df_final.columns = ["Tomato","Strawberry","Potato","Pomegranate","Plum","Pear","Peach","Orange","Lemon","LadyFinger","Cucumber","Corn","ChilliGreen","Cherry","Carrot","CapsicumGreen","Broccoli","Brinjal","Apple","Banana"]

In [525]:
df_final.head()

,Tomato,Strawberry,Potato,Pomegranate,Plum,Pear,Peach,Orange,Lemon,LadyFinger,Cucumber,Corn,ChilliGreen,Cherry,Carrot,CapsicumGreen,Broccoli,Brinjal,Apple,Banana
0,0.341002,0.315255,0.375852,0.292136,0.226278,0.804610,0.394865,0.309855,0.370026,0.237663,0.507296,0.162051,0.491359,0.250759,0.440752,0.580714,0.348162,0.383197,0.525455,0.534317
1,0.537191,0.458828,0.452231,0.318448,0.311758,0.498060,0.416896,0.806128,0.385563,0.552115,0.500968,0.252959,0.410454,0.264638,0.461590,0.356402,0.362346,0.416804,0.444392,0.520542
2,0.386674,0.406583,0.563667,0.379463,0.572559,0.469921,0.362688,0.276406,0.383648,0.456395,0.460107,0.364422,0.535779,0.274900,0.552798,0.535719,0.250374,0.461672,0.504270,0.666670
3,0.335456,0.443824,0.618811,0.360106,0.405951,0.306371,0.328475,0.465907,0.414677,0.951618,0.457385,0.420880,0.565803,0.302666,0.620328,0.502862,0.269386,0.312841,0.373471,0.384275
4,0.372083,0.387127,0.599096,0.349777,0.505208,0.452595,0.428940,0.441945,0.455679,0.711824,0.399981,0.255603,0.297288,0.290908,0.653533,0.509162,0.321159,0.353953,0.288864,0.592705


In [527]:
tfile = open('ssim_.txt', 'a')
tfile.write(df_final.to_string())
tfile.close()